In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader,TensorDataset

import numpy as np
import os
import time
import datetime

torch.manual_seed(1)
np.random.seed(1)
torch.cuda.manual_seed(1)
torch.backends.cudnn.benchmark=True

In [12]:
from dna_data import *

# Hyperparameters

In [3]:
TOTAL_EPOCH = 100
BATCH_SIZE = 100
LEARNING_RATE = 0.01
NUM_WORKERS=0

session_name = '20191228'

# Check CUDA device environmnet

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
print(device,torch.cuda.device_count())

cuda:0 6


# Useful functions and classes

## mkdir function 

In [6]:
def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [7]:
def print_time(trainstart_time):
    log.info(msg=("{:s} | {:.2f}sec elapsed from checkpoint".format(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),time.time()-trainstart_time)))

## Save model

In [8]:
# build model file
model_path = 'model/model%s/'% session_name
mkdir(model_path)

def save_model(path,model,optimizer):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, path)

# Training

## Initialize Dataset Loader

In [13]:
train_dataset_list=[DeepSEA_Dataset('data/train_{}.pt'.format(i),'trainxdata','traindata') for i in range(10)]

KeyboardInterrupt: 

In [10]:
BATCH_SIZE=2000
NUM_WORKERS=4

In [11]:
train_loader_list=[DataLoader(dataset=train_dataset_list[i],shuffle=True,batch_size=BATCH_SIZE,num_workers=NUM_WORKERS) for i in range(10)]

NameError: name 'train_dataset_list' is not defined

In [16]:
valid_loader = DataLoader(
    dataset=DeepSEA_Dataset('data/valid.mat','validxdata','validdata'), 
    shuffle=False,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS
)

## Initialize Model

In [18]:
deepsea = DeepSEA()
if torch.cuda.device_count() > 1:
    log.info(msg=("Let's use", torch.cuda.device_count(), "GPUs!"))
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    deepsea = nn.DataParallel(deepsea)
deepsea.to(device)

 2019-12-28 21:40:41,935 [INFO] (<ipython-input-18-d188840bd447>:3) > ("Let's use", 6, 'GPUs!')


DataParallel(
  (module): DeepSEA(
    (Conv1): Conv1d(4, 320, kernel_size=(8,), stride=(1,))
    (Conv2): Conv1d(320, 480, kernel_size=(8,), stride=(1,))
    (Conv3): Conv1d(480, 960, kernel_size=(8,), stride=(1,))
    (Maxpool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (Drop1): Dropout(p=0.2, inplace=False)
    (Drop2): Dropout(p=0.5, inplace=False)
    (Linear1): Linear(in_features=50880, out_features=925, bias=True)
    (Linear2): Linear(in_features=925, out_features=919, bias=True)
  )
)

In [23]:
"""
checkpoint=torch.load('model/model20191228/deepsea_net_params_epoch0.pt')
deepsea.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
"""

## Define optimizer, scheduler, loss_function, etc.

In [56]:
optimizer = optim.SGD(deepsea.parameters(), lr=LEARNING_RATE,momentum=0.9,weight_decay=5e-07)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5,verbose=1)
loss_function = nn.BCEWithLogitsLoss()

## Train the Model

In [ ]:
trainstart_time = time.time()
check_time=time.time()
for epoch in range(3,TOTAL_EPOCH):
    train_losses = []
    valid_losses = []
    log.info(msg=("Epoch {:d} started".format(epoch)))
    
    """
    Data Training
    """
    deepsea.train()
    train_loss = 0
    for i in range(10):
        log.info(msg=("{:d}th traindata set".format(i)))
        for batch_i,(train_batch_X,train_batch_Y) in enumerate(train_loader_list[i]):
            #log.info(msg=("loading",time.time()-a)
            train_batch_X=train_batch_X.to(device)
            train_batch_Y=train_batch_Y.to(device)

            out=deepsea(train_batch_X)

            """
            L2_reg=0
            for name, param in deepsea.named_parameters():
                if 'weight' in name:
                L2_reg = L2_reg + torch.norm(param, 2)**2
            """
            loss = loss_function(out, train_batch_Y)#+*L2_reg+1e-08
            
            train_losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            #if step%int(len(train_loader.dataset)/BATCH_SIZE/100)==0:
            batch_interval=30
            if batch_i%batch_interval==0:
                print_time(trainstart_time)
                log.info(msg=("sec per data",(time.time()-check_time)/(BATCH_SIZE*batch_interval))); check_time=time.time()
                log.info(msg=("{:d}th batch / {:.2f}% of one epoch".format(batch_i,
                                                                   (i*len(train_loader_list[i].dataset)+BATCH_SIZE*batch_i)/
                                                                   (10*len(train_loader_list[i].dataset)) *100)))
                log.info(msg=("--------------------------"))
            #log.info(msg=("compute",time.time()-a)
            a=time.time()

    """
    Evaluate
    """
        
    deepsea.eval()
    
    for valid_step, (valid_batch_X, valid_batch_Y) in enumerate(valid_loader):
        valid_batch_X = valid_batch_X.to(device)
        valid_batch_Y = valid_batch_Y.to(device)

        val_out = deepsea(valid_batch_X)
        val_loss = loss_function(val_out, valid_batch_Y)
        valid_losses.append(val_loss.item())
        
    train_loss = np.average(train_losses)
    valid_loss = np.average(valid_losses)
    
    scheduler.step(valid_loss)

    print_time(trainstart_time)
    log.info(msg=("epoch:{:d}, train_loss:{:f}, valid_loss:{:f}".format(epoch,train_loss,valid_loss)))
    if epoch % 5 == 0:
        save_model(model_path+'deepsea_net_params_epoch{}.pt'.format(epoch),deepsea,optimizer)
    log.info(msg=("--------------------------"))
    """
    Visdom Data Plotting
    """    
    vis.line(
        X=np.array([epoch]),
        Y=np.array([train_loss]),
        win=win,
        update="append",
        name="train"
    )
    vis.line(
        X=np.array([epoch]),
        Y=np.array([valid_loss]),
        win=win,
        update="append",
        name="val"
    )

In [49]:
torch.norm(torch.Tensor(np.array([1,2,3])),2)**2

tensor(14.0000)

In [36]:
L1_sparse=0
for wefor name, param in deepsea.named_parameters()
    print(name)
    #if 'weight' in name:
    #    L1_reg = L1_reg + torch.norm(param, 1)

module.Conv1.weight
module.Conv1.bias
module.Conv2.weight
module.Conv2.bias
module.Conv3.weight
module.Conv3.bias
module.Linear1.weight
module.Linear1.bias
module.Linear2.weight
module.Linear2.bias


In [32]:
save_model(model_path+'deepsea_net_params_epoch{}.pt'.format(epoch),deepsea,optimizer)